In [1]:
! pip install wikipedia
! pip install nltk
! pip install rake-nltk

In [2]:
import wikipedia
import random
import tqdm
import nltk
import re
import string
import threading
import multiprocessing
import os
import shutil
from nltk.corpus import stopwords

In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dropout, Dense
from keras.layers import LSTM, Bidirectional, BatchNormalization
from keras.layers import SpatialDropout1D, Flatten, Conv2D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.regularizers import l2, l1_l2
import keras

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
wikipedia.set_lang("ru")

In [6]:
def wikiTexts(topic, linksNum:list = [0], first:bool=True) -> (list, list) :
    links = []
    try:
        page = wikipedia.page(topic)
    except:
        return [], []
    
    try:
      links = page.links
    except:
      return result, [topic]
    
    result = [page.content]
    if len(linksNum) == 0:
        return result, [topic]
    
    random.shuffle(links)

    iterate = links[:linksNum[0]]
    resultLinks = [*iterate]
    if first:
      iterate = tqdm.tqdm_notebook(iterate)
    
    for link in iterate:
        contents = wikiTexts(link, linksNum[1:], False)
        result += contents[0]
        resultLinks += contents[1]
    return result, resultLinks

In [7]:
def wikiThread(returnDict, topic, linksNum:list = [0]):
  returnDict[topic] = wikiTexts(topic, linksNum)

In [8]:
depthSearch = [150, 2] # 150 первых ссылок, по 2 вторичных ссылки из каждой первой ссылки
jobs = []

manager = multiprocessing.Manager()
returnDict = manager.dict()

tasks = [
         (returnDict, "животные", depthSearch),
         (returnDict, "музыка", depthSearch),
         (returnDict, "спорт", depthSearch),
         (returnDict, "литература", depthSearch),
]
for i in range(len(tasks)):
  out_list = list()
  process = multiprocessing.Process(target=wikiThread, 
                                    args=tasks[i])
  jobs.append(process)

for j in jobs:
  j.start()

for j in jobs:
  j.join()
animalTexts, animalLinks = returnDict["животные"]
musicTexts, musicLinks = returnDict["музыка"]
sportTexts, sportLinks = returnDict["спорт"]
literatureTexts, literatureLinks = returnDict["литература"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that c

In [9]:
allTexts = (animalTexts, musicTexts, sportTexts, literatureTexts)
allLinks = (animalLinks, musicLinks, sportLinks, literatureLinks)
categories = ("животные", "музыка", "спорт", "литература")

In [10]:
for category, texts, links in zip(categories, allTexts, allLinks):
  print(category, ":", sum([len(text.split(' ')) for text in texts]), "слов")
  print(*links, sep=", ")

животные : 600418 слов
Зелёные растения, Current Biology, Ctenophora, Тихоходки, Malawimonada, Животные в религиозных представлениях и мифах, Естествознание, Microheliella, Жир, Прометазои, Horodyskia, 1758 год в науке, Антропология, Мезозой, Ancyromonadida, Глаукофитовые водоросли, Шестилучевые губки, Оболочники, Первичнополостные черви, Holozoa, Кости, Четвертичный период, Apusomonadida, Щетина, Девонский период, Таксон, Хоаноциты, Юрский период, Ордовикско-силурийское вымирание, Savalia savaglia, Жгутик, Rappemonads, Продукты питания, Xenusia, Кембрийский период, Метамерия (биология), Amorphea, Corallochytrium, Массовое пермское вымирание, Appleton-Century-Crofts, Arthropoda, Линяющие, Пресмыкающиеся, Гнатостомулиды, Страменопилы, Двустворчатые, Одноклеточность, Пластинчатые, Морские звёзды, Protozoa, Гомология (биология), Xenacoelomorpha, Sar, Криптофитовые водоросли, Doi, Латинский язык, Мел-палеогеновое вымирание, Целом, Opisthosporidia, Бесщупальцевые, Амфибии, Род (биология), О

In [11]:
stopWordsRu = set(stopwords.words("russian"))
punctuation = set(string.punctuation + "—")

def validateWord(word):
  if word in stopWordsRu or word in punctuation:
    return False
  allSymValid = False

  for sym in word:
    if not sym in punctuation:
      allSymValid = True
      break

  return allSymValid

def prepareText(text:str) -> list:
  out = []
  for word in nltk.word_tokenize(text):
    word = word.strip()
    if not validateWord(word):
      continue
    out.append(word.lower())
  return out


def generateTexts(texts, windowLen) -> list:
  out = []
  for text in tqdm.tqdm_notebook(texts):
    byWords = text.split(' ')
    for window in range(len(byWords) // windowLen):
      textPrepare = " ".join(byWords[window * windowLen: (window + 1) * windowLen])

      prepared = prepareText(textPrepare)
      if windowLen - len(prepared) > 0:
        prepared += [" "] * ( windowLen - len(prepared))
      out.append(prepared)
  return out

In [12]:
sentMaxLen = 300

In [13]:
animalsPrepared = generateTexts(animalTexts, sentMaxLen)
musicPrepared = generateTexts(musicTexts, sentMaxLen)
sportPrepared = generateTexts(sportTexts, sentMaxLen)
literaturePrepared = generateTexts(literatureTexts, sentMaxLen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [14]:
dataFullX = animalsPrepared + musicPrepared + sportPrepared + literaturePrepared
dataFullY = np.array([[1, 0, 0, 0]] * len(animalsPrepared) + [[0, 1, 0, 0]] * len(musicPrepared) + [[0, 0, 1, 0]] * len(sportPrepared) + [[0, 0, 0, 1]] * len(literaturePrepared))

In [15]:
len(dataFullX), len(dataFullY)

(10635, 10635)

In [16]:
maxWords = 5000
tokenizer = Tokenizer(num_words=maxWords, filters=string.punctuation + '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [17]:
tokenizer.fit_on_texts(map(lambda x: " ".join(x), dataFullX))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 248032 unique tokens.


In [18]:
X = tokenizer.texts_to_sequences(map(lambda x: " ".join(x), dataFullX))
X = pad_sequences(X, maxlen=sentMaxLen)
X.shape

(10635, 300)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dataFullY, test_size = 0.1, shuffle=True)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9571, 300) (9571, 4)
(1064, 300) (1064, 4)


In [20]:
embDim = 128

In [21]:
model = Sequential([
                    Embedding(maxWords, embDim, input_length=X.shape[1]),
                    SpatialDropout1D(0.2),
                    LSTM(64, kernel_regularizer=l1_l2(0.0005)),
                    Dropout(0.2),
                    Dense(Y_test.shape[1], activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [22]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 128)          640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 689,668
Trainable params: 689,668
Non-trainable params: 0
_________________________________________________________________


In [23]:
epochs = 25
batch_size = 512

In [24]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr / (2.7)
history = model.fit(X_train,
                    Y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[
                            keras.callbacks.LearningRateScheduler(scheduler)  
                    ])

Epoch 1/25
17/17 [==============================] - 24s 138ms/step - loss: 3.8843 - accuracy: 0.3022 - val_loss: 3.2146 - val_accuracy: 0.3048
Epoch 2/25
17/17 [==============================] - 1s 73ms/step - loss: 3.0194 - accuracy: 0.3632 - val_loss: 2.4755 - val_accuracy: 0.4405
Epoch 3/25
17/17 [==============================] - 1s 73ms/step - loss: 2.2947 - accuracy: 0.4978 - val_loss: 1.8976 - val_accuracy: 0.4823
Epoch 4/25
17/17 [==============================] - 1s 73ms/step - loss: 1.7371 - accuracy: 0.5526 - val_loss: 1.4727 - val_accuracy: 0.5887
Epoch 5/25
17/17 [==============================] - 1s 74ms/step - loss: 1.3364 - accuracy: 0.6429 - val_loss: 1.2249 - val_accuracy: 0.6127
Epoch 6/25
17/17 [==============================] - 1s 74ms/step - loss: 1.0410 - accuracy: 0.7205 - val_loss: 1.0324 - val_accuracy: 0.6545
Epoch 7/25
17/17 [==============================] - 1s 73ms/step - loss: 0.8644 - accuracy: 0.7468 - val_loss: 0.9408 - val_accuracy: 0.7004
Epoch 8/25


In [25]:
model.evaluate(X_test, Y_test)

34/34 [==============================] - 0s 10ms/step - loss: 0.9069 - accuracy: 0.7180


[0.9068658351898193, 0.7180451154708862]

In [1]:
import pandas as pd

In [2]:
testDf = pd.read_csv('test.csv')

In [3]:
testDf.head(5)

,id,task
0,0,Вопрос 1 Денис готовится провести соревнования...
1,1,Реши задачу.\nСколько коробок корма для собак ...
2,2,"Если групп элементов несколько, то применяется..."
3,3,Вопрос1 ...
4,4,Для наглядной геометрической иллюстрации объём...


In [20]:
testDf['task'].values[1]

'Реши задачу.\nСколько коробок корма для собак поместится в ящике, если все стороны ящика соответственно в 12 раз больше сторон коробки корма?'

In [29]:
Xtest = tokenizer.texts_to_sequences(map(lambda x: str(x['task']), testDf.iloc))
Xtest = pad_sequences(Xtest, maxlen=sentMaxLen)

In [30]:
predictions = model.predict(Xtest)
predictions

array([[0.09735209, 0.43359113, 0.16346458, 0.3055922 ],
       [0.07233533, 0.48948306, 0.1018154 , 0.33636627],
       [0.1266391 , 0.41387922, 0.18310064, 0.27638113],
       ...,
       [0.06528526, 0.3648363 , 0.1388722 , 0.43100628],
       [0.22779113, 0.59625584, 0.09307627, 0.08287673],
       [0.06749827, 0.5344242 , 0.12004167, 0.27803588]], dtype=float32)

In [31]:
predictions = list(map(lambda x: categories[int(x)], np.argmax(predictions, axis=-1)))
predictions[:5]

['музыка', 'музыка', 'музыка', 'музыка', 'музыка']

In [32]:
from rake_nltk import Rake

def getKeyWords(textSeries: pd.Series, threshold=4, minLen=1, maxLen=2):
    r = Rake(
        stopwords=stopWordsRu,
        punctuations=punctuation,
        min_length=1, max_length=2
    )
    results = []
    for i in range(textSeries.shape[0]):
      r.extract_keywords_from_text(testDf['task'].iloc[i])
      rowResult = r.get_ranked_phrases_with_scores()
      selected = 1
      while selected < len(rowResult):
        if rowResult[selected][0] < threshold:
          break
        selected += 1
      results.append(rowResult[:selected])
    return list(map(lambda task: list(map(lambda keyword: keyword[1], task)), results))

In [33]:
keyWords = getKeyWords(testDf['task'])

In [34]:
keyWordsReady = list(map( lambda x: ";".join(x), keyWords))

In [35]:
sample = pd.read_csv('sample_submission.csv')


In [36]:
sample['keywords'] = keyWordsReady

In [37]:
sample['category'] = predictions

In [38]:
sample.to_csv('submit.csv', index=False)

In [39]:
sample['category'].value_counts()

музыка        353
литература    101
спорт          46
животные       14
Name: category, dtype: int64

In [40]:
sample

,id,category,keywords
0,0,музыка,настольному хоккею;вариантов выбора
1,1,музыка,собак поместится;реши задачу
2,2,музыка,течение года;сдавали зачет;реши задачу;равно и...
3,3,музыка,суммы затрат;продукты питания;конце месяца;дом...
4,4,музыка,этими объектами;увлекающихся футболом;либо пон...
...,...,...,...
509,509,музыка,следующие вопросы;проанализируй диаграмму
510,510,музыка,сборке тренажеров;выполнить задание;6 дней;4 д...
511,511,литература,7 дней;1 день
512,512,музыка,ухватил пернатого;улетающую птицу;спрыгнул вни...
